<center> <h1>Αναγνώριση Προτύπων - 2η Εργαστηριακή Άσκηση</h1> </center>

---

<center> <h2>Θέμα: Αναγνώριση φωνής με Κρυφά Μαρκοβιανά Μοντέλα και Αναδρομικά Νευρωνικά Δίκτυα</h2> </center>
<center> <h2>ΣΧΟΛΗ: ΣΗΜΜΥ</h2> </center>
<img src="pyrforos-digamma.png" width="100">

Ονοματεπώνυμο | Αριθμός Μητρώου
------------ | -------------
Γιάννης Πιτόσκας | 03115077
Αντώνης Παπαοικονόμου | 03115140

In [2]:
!pip3 install --upgrade pip
!pip3 install scikit-learn --upgrade
!pip3 install numpy --upgrade
!pip3 install --upgrade joblib
!pip3 install librosa
!pip3 install pomegranate

Requirement already up-to-date: pip in c:\python36\lib\site-packages (19.3.1)
Requirement already up-to-date: scikit-learn in c:\python36\lib\site-packages (0.22)
  Found existing installation: numpy 1.17.4
    Uninstalling numpy-1.17.4:
      Successfully uninstalled numpy-1.17.4


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\python36\\lib\\site-packages\\~umpy\\.libs\\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



Requirement already up-to-date: joblib in c:\python36\lib\site-packages (0.14.1)
    Running setup.py install for audioread: started
    Running setup.py install for audioread: finished with status 'done'
    Running setup.py install for resampy: started
    Running setup.py install for resampy: finished with status 'done'
    Running setup.py install for pycparser: started
    Running setup.py install for pycparser: finished with status 'done'
    Running setup.py install for librosa: started
    Running setup.py install for librosa: finished with status 'done'


## Βήμα 9

In [6]:
import numpy as np
import librosa
from glob import glob
import os

from sklearn.preprocessing import StandardScaler

def parser(directory):
    # Parse relevant dataset info
    print(directory)
    files = glob(os.path.join(directory, '*.wav'))
    print(files[0], files[1])
    fnames = [f.split('/')[1].split('.')[0].split('_') for f in files]
    print(fnames[0], fnames[1])
    ids = [f[2] for f in fnames]
    y = [int(f[0]) for f in fnames]
    speakers = [f[1] for f in fnames]
    _, Fs = librosa.core.load(files[0], sr=None)

    def read_wav(f):
        global Fs
        wav, fs = librosa.core.load(f, sr=None)
        return wav

    # Read all wavs
    wavs = [read_wav(f) for f in files]

    # Extract MFCCs for all wavs
    window = 30 * Fs // 1000
    step = window // 2
    frames = [librosa.feature.mfcc(wav, Fs, n_fft=window, hop_length=window - step, n_mfcc=6).T for wav in wavs]
    # Print dataset info
    print('Total wavs: {}'.format(len(frames)))

    # Standardize data
    scaler = StandardScaler()
    scaler.fit(np.concatenate(frames))
    for i in range(len(frames)):
        frames[i] = scaler.transform(frames[i])

    # Split to train-test
    X_train, y_train, spk_train = [], [], []
    X_test, y_test, spk_test = [], [], []
    test_indices = ['0', '1', '2', '3', '4']
    for idx, frame, label, spk in zip(ids, frames, y, speakers):
        if str(idx) in test_indices:
            X_test.append(frame)
            y_test.append(label)
       ## Βήμα 9     spk_test.append(spk)
       ## Βήμα 9 else:
            X_train.append(frame)
            y_train.append(label)
            spk_train.append(spk)

    return X_train, X_test, y_train, y_test, spk_train, spk_test

record_dir = os.path.abspath("./free-spoken-digit-dataset-master/recordings/")
# os.chdir('./free-spoken-digit-dataset-master/recordings')
X, X_test, y, y_test, spk_train, spk_test = parser(record_dir)
# os.chdir('../../')

X = np.array(X)
y = np.array(y)
X_test = np.array(X_test)
y_test = np.array(y_test)

G:\Documents\Αντώνης\HMMY\ΡΟΗ_Σ\Αναγνώριση_Προτύπων\Lab_2\free-spoken-digit-dataset-master\recordings
G:\Documents\Αντώνης\HMMY\ΡΟΗ_Σ\Αναγνώριση_Προτύπων\Lab_2\free-spoken-digit-dataset-master\recordings\0_jackson_0.wav G:\Documents\Αντώνης\HMMY\ΡΟΗ_Σ\Αναγνώριση_Προτύπων\Lab_2\free-spoken-digit-dataset-master\recordings\0_jackson_1.wav


IndexError: list index out of range

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)


In [ ]:
from pomegranate import *

X_digit = {digit : np.concatenate(X_train[y_train==digit]) for digit in range(10)}

X0 = X[0]


In [ ]:
n_states = 5 # the number of HMM states
n_mixtures = 2 # the number of Gaussians

dists = [] # list of probability distributions for the HMM states
for i in range(n_states):
    a = GeneralMixtureModel.from_samples(MultivariateGaussianDistribution, n_mixtures, X0)
    dists.append(a)
    
trans_mat = np.diag(np.ones(n_states-1), k=1) # your transition matrix
starts = [1] + [0]*(n_states-1) # your starting probability matrix
ends = [0]*(n_states-1) + [1] # your ending probability matrix



data = [] # your data: must be a Python list that contains: 2D lists with the sequences (so its dimension would be num_sequences x seq_length x feature_dimension)
          # But be careful, it is not a numpy array, it is a Python list (so each sequence can have different length)
